## REad the data

In [1]:
import pandas as pd
from sodapy import Socrata
import numpy as np

In [2]:

import pandas as pd
from sodapy import Socrata

client = Socrata("data.texas.gov", None)

results = client.get("ubdr-4uff", limit=300000)

# Convert to pandas DataFrame
df1 = pd.DataFrame.from_records(results)

In [3]:
df = df1

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259342 entries, 0 to 259341
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   complaint_number          259342 non-null  object
 1   respondent_name           259342 non-null  object
 2   complainant_role          259342 non-null  object
 3   reason                    259336 non-null  object
 4   complaint_confirmed_code  259338 non-null  object
 5   disposition               258304 non-null  object
 6   received_date             259342 non-null  object
 7   closed_date               259342 non-null  object
 8   complaint_type            259341 non-null  object
 9   coverage_type             259342 non-null  object
 10  coverage_level            259342 non-null  object
 11  respondent_id             259342 non-null  object
 12  respondent_role           259340 non-null  object
 13  respondent_type           259342 non-null  object
 14  comp

In [5]:
# Drop missing rows in disposition coluMN
df = df.dropna(subset=['disposition'])

In [6]:

# get counts of disposition values in descending order
disposition_counts = df['disposition'].value_counts().sort_values(ascending=False)
print(disposition_counts)

# write disposition counts to a csv file
disposition_counts.to_csv('disposition_counts.csv')

disposition
Contract Language/Legal Issue                                                    25698
No Jurisdiction                                                                  23533
Information Furnished                                                            17924
Question of Fact                                                                 12215
Information Furnished; Contract Language/Legal Issue                              8095
                                                                                 ...  
Claim Settled; Corrective Action Taken; Insufficient Information                     1
Question of Fact; Policy Issued/Restored; Information Furnished                      1
Rate Problem Resolved; Corrective Action Taken; Contract Language/Legal Issue        1
Agent Resp Recvd; Failure to Timely Respond; Corrective Action Taken                 1
Claim Settled; Company Position Upheld; Additional Monies Received                   1
Name: count, Length: 4043, dtyp

### Disposition column

In [7]:


# Define keywords for categorizing dispositions as 'verdict won' or 'verdict not won'
verdict_won_keywords = ["Claim Settled", "Additional Monies Received"]
verdict_not_won_keywords = ["No Jurisdiction", "Company Position Upheld", "Insufficient Information", "Contract Language/Legal Issue", "Question of Fact"]

# Function to categorize each disposition based on the presence of keywords
def categorize_verdict(disposition):
    if any(keyword in disposition for keyword in verdict_won_keywords):
        return "Yes"
    elif any(keyword in disposition for keyword in verdict_not_won_keywords):
        return "No"
    else:
        return "Uncategorized"  # For entries that don't match any keywords

# Apply categorization to the disposition data
df['verdict_passed'] = df['disposition'].apply(categorize_verdict)

# Drop rows where the disposition was categorized as "Uncategorized"
df = df[df['verdict_passed'] != "Uncategorized"]

# replace disposition column with verdict_passed column and drop verdict_passed
df['disposition'] = df['verdict_passed']
df = df.drop(columns=['verdict_passed'])

# Save the modified data back to a CSV file if needed
df.to_csv('categorized_df.csv', index=False)

C:\Users\nbngc\AppData\Local\Temp\ipykernel_68392\3181898158.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['verdict_passed'] = df['disposition'].apply(categorize_verdict)


In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 206708 entries, 4 to 259341
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   complaint_number          206708 non-null  object
 1   respondent_name           206708 non-null  object
 2   complainant_role          206708 non-null  object
 3   reason                    206707 non-null  object
 4   complaint_confirmed_code  206704 non-null  object
 5   disposition               206708 non-null  object
 6   received_date             206708 non-null  object
 7   closed_date               206708 non-null  object
 8   complaint_type            206707 non-null  object
 9   coverage_type             206708 non-null  object
 10  coverage_level            206708 non-null  object
 11  respondent_id             206708 non-null  object
 12  respondent_role           206708 non-null  object
 13  respondent_type           206708 non-null  object
 14  complaina

# Drop unnecessary columns

In [9]:
# drop the columns
# count of individual in column respondant type
df['respondent_type'].value_counts()

# drop columns keyword, respondent_id,complaint_number
df = df.drop(columns=['keyword','respondent_id','complaint_number'])
 

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 206708 entries, 4 to 259341
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   respondent_name           206708 non-null  object
 1   complainant_role          206708 non-null  object
 2   reason                    206707 non-null  object
 3   complaint_confirmed_code  206704 non-null  object
 4   disposition               206708 non-null  object
 5   received_date             206708 non-null  object
 6   closed_date               206708 non-null  object
 7   complaint_type            206707 non-null  object
 8   coverage_type             206708 non-null  object
 9   coverage_level            206708 non-null  object
 10  respondent_role           206708 non-null  object
 11  respondent_type           206708 non-null  object
 12  complainant_type          206708 non-null  object
 13  involved_party_type       186269 non-null  object
dtypes: object

In [11]:
# Trim whitespace from all columns with string data
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [12]:
# Convert all columns to propercase if they are strings
df = df.apply(lambda x: x.str.title() if x.dtype == "object" else x)

In [13]:
df.head()

,respondent_name,complainant_role,reason,complaint_confirmed_code,disposition,received_date,closed_date,complaint_type,coverage_type,coverage_level,respondent_role,respondent_type,complainant_type,involved_party_type
4,"Charter Oak Fire Insurance Company, The",Insured,Unsatisfactory Settle/Offer,No,No,2012-06-13T00:00:00.000,2012-07-17T00:00:00.000,Property And Casualty,Automobile,Individual Private Pass,Ins Co - Licensed/Active,Organization,Indv,NaN
5,Reassure America Life Insurance Company,Insured,Cash Value; Misrepresentation,No,No,2012-06-18T00:00:00.000,2012-07-30T00:00:00.000,"Life, Accident And Health",Life & Annuity,Individual Life,Ins Co - Licensed/Active,Organization,Indv,NaN
6,Usaa General Indemnity Company,Third Party,Unsatisfactory Settle/Offer,No,Yes,2012-02-27T00:00:00.000,2012-07-02T00:00:00.000,Property And Casualty,Automobile,Individual Private Pass,Ins Co - Licensed/Active,Organization,Indv,Insured
7,"Blue Cross And Blue Shield Of Texas, A Divisio...",Insured,Denial Of Claim,No,No,2012-06-19T00:00:00.000,2012-08-06T00:00:00.000,"Life, Accident And Health",Accident And Health,Group A&H,Ins Co - Licensed/Active,Organization,Indv,NaN
8,Property And Casualty Insurance Company Of Har...,Insured,Unsatisfactory Settle/Offer,No,No,2011-10-03T00:00:00.000,2012-07-20T00:00:00.000,Property And Casualty,Homeowners,Homeowners,Ins Co - Licensed/Active,Organization,Indv,NaN


In [14]:
#list unique values in involved party type
import csv
df['involved_party_type'].unique()
# write unique values to a separate csv file
with open('involved_party_type.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in df['involved_party_type'].unique():
        writer.writerow([item])


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 206708 entries, 4 to 259341
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   respondent_name           206708 non-null  object
 1   complainant_role          206708 non-null  object
 2   reason                    206707 non-null  object
 3   complaint_confirmed_code  206704 non-null  object
 4   disposition               206708 non-null  object
 5   received_date             206708 non-null  object
 6   closed_date               206708 non-null  object
 7   complaint_type            206707 non-null  object
 8   coverage_type             206708 non-null  object
 9   coverage_level            206708 non-null  object
 10  respondent_role           206708 non-null  object
 11  respondent_type           206708 non-null  object
 12  complainant_type          206708 non-null  object
 13  involved_party_type       186269 non-null  object
dtypes: object

In [16]:
df.to_csv('categorized_df.csv', index=False)

# respondant name

In [17]:
# Remove the text ", The" from respondent_name column if ends in ", The"
df['respondent_name'] = df['respondent_name'].str.rstrip(', The')
# Perform target encoding on respondent_name column
from sklearn.preprocessing import LabelEncoder
# Initialize the LabelEncoder
le = LabelEncoder()
# Fit and transform the respondent_name column
df['respondent_name_encoded'] = le.fit_transform(df['respondent_name'])
